# Indian Takeaway Orders
---
## By Killian Hughes

### Load in data

In [0]:
restaurant_1_orders = sqlContext.read.load('/FileStore/tables/restaurant_1_orders.csv', format='csv', header=True, inferSchema=True)
restaurant_2_orders = sqlContext.read.load('/FileStore/tables/restaurant_2_orders.csv', format='csv', header=True, inferSchema=True)
restaurant_1_products_price = sqlContext.read.load('/FileStore/tables/restaurant_1_products_price.csv', format='csv', header=True, inferSchema=True)
restaurant_2_products_price = sqlContext.read.load('/FileStore/tables/restaurant_2_products_price.csv', format='csv', header=True, inferSchema=True)

### Data Cleaning
---
  - Using Distinct Count and Count to check for duplicates
  - Using dropDuplicates to remove duplicates

In [0]:
def dup_percent_finder(dataframe):
    percent_dup = ((dataframe.count() - dataframe.distinct().count()) / dataframe.count()) * 100
    print("This dataframe contains", percent_dup, "% duplicated rows")

In [0]:
dup_percent_finder(restaurant_1_orders)

This dataframe contains 0.1296479456815205 % duplicated rows


In [0]:
dup_percent_finder(restaurant_1_products_price)

This dataframe contains 0.0 % duplicated rows


In [0]:
dup_percent_finder(restaurant_2_orders)

This dataframe contains 0.14935015899918613 % duplicated rows


In [0]:
dup_percent_finder(restaurant_2_products_price)

This dataframe contains 0.0 % duplicated rows


In [0]:
restaurant_1_orders = restaurant_1_orders.dropDuplicates()
restaurant_2_orders = restaurant_2_orders.dropDuplicates()

In [0]:
dup_percent_finder(restaurant_1_orders)

This dataframe contains 0.0 % duplicated rows


In [0]:
dup_percent_finder(restaurant_2_orders)

This dataframe contains 0.0 % duplicated rows


In [0]:
import numpy as np
from pyspark.sql.functions import col, isnan, when, count

def null_value_finder(dataframe):
    dataframe.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in dataframe.columns]
    ).show()

In [0]:
restaurant_1_orders.show()

+------------+----------------+--------------------+--------+-------------+--------------+
|Order Number|      Order Date|           Item Name|Quantity|Product Price|Total products|
+------------+----------------+--------------------+--------+-------------+--------------+
|       16115|03/08/2019 20:01|       Plain Papadum|       1|          0.8|             7|
|       16117|03/08/2019 20:17|       Mushroom Rice|       1|         3.95|             7|
|       16116|03/08/2019 20:09|          Aloo Chaat|       1|         4.95|             5|
|       16115|03/08/2019 20:01|      Chicken Pakora|       1|         5.95|             7|
|       16118|03/08/2019 20:25|       Mushroom Rice|       1|         3.95|             6|
|       16118|03/08/2019 20:25|       Mango Chutney|       1|          0.5|             6|
|       16118|03/08/2019 20:25|         Garlic Naan|       1|         2.95|             6|
|       16116|03/08/2019 20:09|      Chicken Pakora|       1|         5.95|             5|

In [0]:
restaurant_2_orders.show()

+--------+----------------+--------------------+--------+-------------+--------------+
|Order ID|      Order Date|           Item Name|Quantity|Product Price|Total products|
+--------+----------------+--------------------+--------+-------------+--------------+
|   25582|03/08/2019 20:33|       Peshwari Naan|       1|         2.95|             5|
|   25583|03/08/2019 21:58|             Paratha|       1|         2.95|            12|
|   25582|03/08/2019 20:33|         Bottle Coke|       1|         2.95|             5|
|   25583|03/08/2019 21:58|Tandoori Mixed Grill|       1|        11.95|            12|
|   25583|03/08/2019 21:58|       Plain Papadum|       1|          0.8|            12|
|   25583|03/08/2019 21:58|         Garlic Naan|       1|         2.95|            12|
|   25583|03/08/2019 21:58|        Madras Sauce|       1|         3.95|            12|
|   25582|03/08/2019 20:33|          Pilau Rice|       1|         2.95|             5|
|   25581|03/08/2019 20:01|          Pilau 

In [0]:
null_value_finder(restaurant_1_orders)

+------------+----------+---------+--------+-------------+--------------+
|Order Number|Order Date|Item Name|Quantity|Product Price|Total products|
+------------+----------+---------+--------+-------------+--------------+
|           0|         0|        0|       0|            0|             0|
+------------+----------+---------+--------+-------------+--------------+



In [0]:
null_value_finder(restaurant_2_orders)

+--------+----------+---------+--------+-------------+--------------+
|Order ID|Order Date|Item Name|Quantity|Product Price|Total products|
+--------+----------+---------+--------+-------------+--------------+
|       0|         0|        0|       0|            0|             0|
+--------+----------+---------+--------+-------------+--------------+



### finding discrepencies between orders and menu from restaurant 1

In [0]:
actual_menu_1 = restaurant_1_orders.select("Item Name", "Product Price")
expected_menu_1 = restaurant_1_products_price.select("Item Name", "Product Price")

In [0]:
actual_menu_1 = actual_menu_1.dropDuplicates()
expected_menu_1 = expected_menu_1.dropDuplicates()

In [0]:
extra_items = actual_menu_1.subtract(expected_menu_1)
extra_items = extra_items.withColumnRenamed("Product Price", "Mistaken Price")
extra_items.show()

+-------------+--------------+
|    Item Name|Mistaken Price|
+-------------+--------------+
|        Rogon|          9.95|
|        Bhuna|          9.95|
|         Saag|         12.95|
|      Dhansak|         12.95|
|        Korma|          9.95|
|     Mushroom|          9.95|
|      Dupiaza|          9.95|
|Chicken Tikka|          7.95|
|       Madras|         11.95|
|        Bhuna|         12.95|
|         Saag|          9.95|
|      Dhansak|          9.95|
|        Curry|         11.95|
|      Dupiaza|         12.95|
|  Sheek Kehab|          5.95|
|     Vindaloo|         11.95|
|        Curry|          8.95|
|Chicken Tikka|          4.95|
|     Vindaloo|          8.95|
|       Pathia|          9.95|
+-------------+--------------+
only showing top 20 rows



In [0]:
expected_menu = expected_menu_1.withColumnRenamed("Item Name", "Drop").withColumnRenamed("Product Price", "Actual Price")
expected_menu.show()

+-------------+------------+
|         Drop|Actual Price|
+-------------+------------+
|   Plain Rice|        2.95|
|   Onion Naan|        2.95|
|  Lime Pickle|         0.5|
|        Puree|        1.95|
|Peshwari Naan|        2.95|
|Mango Chutney|         0.5|
|   Pilau Rice|        2.95|
|   Mint Sauce|         0.5|
|  Bottle Coke|        2.95|
|    Red Sauce|         0.5|
|  Green Salad|        2.95|
|Tandoori Roti|         2.6|
|      Chapati|        1.95|
|   Keema Naan|        2.95|
|      Paratha|        2.95|
|Onion Chutney|         0.5|
|Spicy Papadum|         0.8|
|   Plain Naan|         2.6|
|       Raitha|        2.95|
|Plain Papadum|         0.8|
+-------------+------------+
only showing top 20 rows



In [0]:
df1 = extra_items.join(expected_menu, extra_items["Item Name"] == expected_menu["Drop"], "inner")
df1.show()

+-------------+--------------+-------------+------------+
|    Item Name|Mistaken Price|         Drop|Actual Price|
+-------------+--------------+-------------+------------+
|       Pathia|         12.95|       Pathia|        8.95|
|        Rogon|          9.95|        Rogon|        8.95|
|        Bhuna|          9.95|        Bhuna|        8.95|
|         Saag|         12.95|         Saag|        8.95|
|      Dhansak|         12.95|      Dhansak|        8.95|
|        Korma|          9.95|        Korma|        8.95|
|     Mushroom|          9.95|     Mushroom|        8.95|
|      Dupiaza|          9.95|      Dupiaza|        8.95|
|Chicken Tikka|          7.95|Chicken Tikka|        3.95|
|       Madras|         11.95|       Madras|        7.95|
|        Korma|         12.95|        Korma|        8.95|
|        Bhuna|         12.95|        Bhuna|        8.95|
|         Saag|          9.95|         Saag|        8.95|
|      Dhansak|          9.95|      Dhansak|        8.95|
|        Curry

In [0]:
df1 = df1.drop("Drop")

In [0]:
display(df1)

Item Name,Mistaken Price,Actual Price
Pathia,12.95,8.95
Rogon,9.95,8.95
Bhuna,9.95,8.95
Saag,12.95,8.95
Dhansak,12.95,8.95
Korma,9.95,8.95
Mushroom,9.95,8.95
Dupiaza,9.95,8.95
Chicken Tikka,7.95,3.95
Madras,11.95,7.95


### removing the mistaken item prices replacing them with the correct prices

In [0]:
restaurant_1_orders
df1

Out[22]: DataFrame[Item Name: string, Mistaken Price: double, Actual Price: double]

In [0]:
restaurant_1_orders.columns

Out[23]: ['Order Number',
 'Order Date',
 'Item Name',
 'Quantity',
 'Product Price',
 'Total products']

In [0]:
restaurant_1_orders.show()

+------------+----------------+--------------------+--------+-------------+--------------+
|Order Number|      Order Date|           Item Name|Quantity|Product Price|Total products|
+------------+----------------+--------------------+--------+-------------+--------------+
|       16115|03/08/2019 20:01|       Plain Papadum|       1|          0.8|             7|
|       16117|03/08/2019 20:17|       Mushroom Rice|       1|         3.95|             7|
|       16116|03/08/2019 20:09|          Aloo Chaat|       1|         4.95|             5|
|       16115|03/08/2019 20:01|      Chicken Pakora|       1|         5.95|             7|
|       16118|03/08/2019 20:25|       Mushroom Rice|       1|         3.95|             6|
|       16118|03/08/2019 20:25|       Mango Chutney|       1|          0.5|             6|
|       16118|03/08/2019 20:25|         Garlic Naan|       1|         2.95|             6|
|       16116|03/08/2019 20:09|      Chicken Pakora|       1|         5.95|             5|

  - join the menu prices with the actual prices for both tables
  - find out the most popular items in each category
  - find the most popular time of day, using the between command

In [0]:
restaurant_1_products_price = restaurant_1_products_price.withColumnRenamed("Item Name", "Menu_Item").withColumnRenamed("Product Price", "Correct_Price")

In [0]:
joined_order_1 = restaurant_1_orders.join(restaurant_1_products_price, restaurant_1_orders["Item Name"] == restaurant_1_products_price["Menu_Item"], "inner")

In [0]:
joined_order_1 = joined_order_1.drop("Product Price", "Item Name")

In [0]:
joined_order_1.orderBy("Order Number", ascending=True).show()

+------------+----------------+--------+--------------+--------------------+-------------+
|Order Number|      Order Date|Quantity|Total products|           Menu_Item|Correct_Price|
+------------+----------------+--------+--------------+--------------------+-------------+
|         630|01/09/2015 14:42|       1|             3|         Onion Bhaji|         3.95|
|         647|08/09/2015 17:12|       1|             3|         Onion Bhaji|         3.95|
|         648|08/09/2015 17:31|       1|             1|         Onion Bhaji|         3.95|
|         651|08/09/2015 17:56|       1|             4|         Onion Bhaji|         3.95|
|         764|09/09/2015 09:31|       1|             3|         Onion Bhaji|         3.95|
|         838|09/09/2015 11:28|       1|             4|         Onion Bhaji|         3.95|
|         840|09/09/2015 11:35|       1|             3|         Onion Bhaji|         3.95|
|         857|09/09/2015 12:48|       1|             4|         Onion Bhaji|         3.95|

### Repeating the same processes for restaurant two


In [0]:
restaurant_2_orders.show()

+--------+----------------+--------------------+--------+-------------+--------------+
|Order ID|      Order Date|           Item Name|Quantity|Product Price|Total products|
+--------+----------------+--------------------+--------+-------------+--------------+
|   25582|03/08/2019 20:33|       Peshwari Naan|       1|         2.95|             5|
|   25583|03/08/2019 21:58|             Paratha|       1|         2.95|            12|
|   25582|03/08/2019 20:33|         Bottle Coke|       1|         2.95|             5|
|   25583|03/08/2019 21:58|Tandoori Mixed Grill|       1|        11.95|            12|
|   25583|03/08/2019 21:58|       Plain Papadum|       1|          0.8|            12|
|   25583|03/08/2019 21:58|         Garlic Naan|       1|         2.95|            12|
|   25583|03/08/2019 21:58|        Madras Sauce|       1|         3.95|            12|
|   25582|03/08/2019 20:33|          Pilau Rice|       1|         2.95|             5|
|   25581|03/08/2019 20:01|          Pilau 

In [0]:
actual_orders_2 = restaurant_2_orders.select("Item Name", "Product Price")
expected_menu_2 = restaurant_2_products_price.select("Item Name", "Product Price")
actual_orders_2 = actual_orders_2.dropDuplicates()
expected_menu_2 = expected_menu_2.dropDuplicates()

In [0]:
mistaken_items_2 = actual_orders_2.subtract(expected_menu_2)
mistaken_items_2 = mistaken_items_2.withColumnRenamed("Product Price", "Mistaken Price")
mistaken_items_2.show()

+------------------+--------------+
|         Item Name|Mistaken Price|
+------------------+--------------+
|   Vindaloo - lamb|          7.95|
|Vindaloo - chicken|          7.95|
|     Madras - lamb|          7.95|
|  Madras - chicken|          7.95|
|      Curry - lamb|          7.95|
|   Curry - chicken|          7.95|
|      Korma - lamb|          8.95|
|      Bhuna - lamb|          8.95|
|  Pathia - chicken|          8.95|
|   Cylon - chicken|          8.95|
|     Curry - prawn|          8.95|
|Mushroom - chicken|          8.95|
|   Mushroom - lamb|          8.95|
|   Methi - chicken|          8.95|
|    Dupiaza - lamb|          8.95|
|    Dhansak - lamb|          8.95|
|       Saag - lamb|          8.95|
| Dupiaza - chicken|          8.95|
|   Rogon - chicken|          8.95|
|      Rogon - lamb|          8.95|
+------------------+--------------+
only showing top 20 rows



In [0]:
expected_menu_2 = expected_menu_2.withColumnRenamed("Item Name", "Drop").withColumnRenamed("Product Price", "Actual Price")
df2 = mistaken_items_2.join(expected_menu_2, mistaken_items_2["Item Name"] == expected_menu_2["Drop"], "inner")
df2 = df2.drop("Drop")
display(df2)

Item Name,Mistaken Price,Actual Price
Mushroom,12.95,8.95
Pathia,12.95,8.95
Rogon,9.95,8.95
Bhuna,9.95,8.95
Saag,12.95,8.95
Dhansak,12.95,8.95
Korma,9.95,8.95
Dupiaza,9.95,8.95
Madras,11.95,7.95
Korma,12.95,8.95


In [0]:
restaurant_2_products_price = restaurant_2_products_price.withColumnRenamed("Item Name", "Menu_Item").withColumnRenamed("Product Price", "Correct_Price")
joined_order_2 = restaurant_2_orders.join(restaurant_2_products_price, restaurant_2_orders["Item Name"] == restaurant_2_products_price["Menu_Item"], "inner")
joined_order_2 = joined_order_2.drop("Product Price", "Item Name") 
joined_order_2.orderBy("Order ID", ascending=True).show()

+--------+----------------+--------+--------------+--------------------+-------------+
|Order ID|      Order Date|Quantity|Total products|           Menu_Item|Correct_Price|
+--------+----------------+--------+--------------+--------------------+-------------+
|    2096|16/12/2015 19:14|       1|             8|Chicken Tikka Jal...|         8.95|
|    2096|16/12/2015 19:14|       1|             8|          Plain Rice|         2.95|
|    2096|16/12/2015 19:14|       1|             8|         Kulcha Naan|         2.95|
|    2096|16/12/2015 19:14|       1|             8|           Aloo Gobi|         5.95|
|    2096|16/12/2015 19:14|       1|             8|          Keema Naan|         2.95|
|    2096|16/12/2015 19:14|       1|             8|         Bombay Aloo|         5.95|
|    2096|16/12/2015 19:14|       1|             8|       Peshwari Naan|         2.95|
|    4487|27/05/2016 16:42|       1|            10|          Mint Sauce|          0.5|
|    4487|27/05/2016 16:42|       1|       

### Data Exploration and Analysis - Which Curries, Startes and Drinks are most popular at each restaurant


In [0]:
joined_order_2 = joined_order_2.withColumn("Total_Paid", joined_order_2.Quantity * joined_order_2.Correct_Price)
joined_order_2.orderBy("Total_Paid", ascending=False).show()
joined_order_1 = joined_order_1.withColumn("Total_Paid", joined_order_1.Quantity * joined_order_1.Correct_Price)
joined_order_1.orderBy("Total_Paid", ascending=False).show()

+--------+----------------+--------+--------------+--------------------+-------------+------------------+
|Order ID|      Order Date|Quantity|Total products|           Menu_Item|Correct_Price|        Total_Paid|
+--------+----------------+--------+--------------+--------------------+-------------+------------------+
|   24419|08/06/2019 16:22|       7|            11| King Prawn Shaslick|        12.95| 90.64999999999999|
|   13900|07/01/2018 15:57|       7|             1|  Sheek Kebab (Main)|         8.95| 62.64999999999999|
|   23349|19/04/2019 19:25|       6|             4|Chicken Tikka (Main)|         8.95|53.699999999999996|
|   18847|17/09/2018 14:55|       5|             1|     Chicken Biryani|         9.95|             49.75|
|    5410|02/09/2016 17:35|       4|             2|Tandoori Mixed Grill|        11.95|              47.8|
|    8256|18/02/2017 20:52|       4|             3|Tandoori Mixed Grill|        11.95|              47.8|
|   19994|09/11/2018 18:27|       4|          

- problems with restaurant 1, orders of 51 King prawn shashlicks, 6769 has ordered 130 curries. Removal of some Order Numbers is in order. 

In [0]:
from pyspark.sql.functions import sum
joined_order_2.groupBy("Order ID").agg(sum("Total_Paid").alias("Overall Paid By Order ID")).orderBy("Overall Paid By Order ID", ascending = False).show()
## Order IDs with the most money spent in restaurant 2

+--------+------------------------+
|Order ID|Overall Paid By Order ID|
+--------+------------------------+
|    7952|                   283.3|
|   13246|                   234.5|
|   19569|      183.65000000000003|
|   22328|      182.35000000000002|
|   14224|                  177.55|
|   24856|      175.95000000000002|
|   21984|                  171.05|
|    5170|      167.60000000000002|
|   24855|                   167.0|
|   24419|                   161.1|
|   19962|      160.60000000000002|
|   11656|                  158.45|
|   17705|      158.20000000000002|
|    5429|                   158.0|
|    9511|                  157.05|
|   24878|                  155.25|
|   19563|      154.35000000000002|
|   21228|      151.10000000000002|
|   17228|                   149.7|
|   16495|                  149.05|
+--------+------------------------+
only showing top 20 rows



In [0]:
joined_order_2.groupBy("Menu_Item").agg(sum("Total_Paid").alias("Total Paid for each Menu Item")).orderBy("Total Paid for each Menu Item", ascending = False).show()

+--------------------+-----------------------------+
|           Menu_Item|Total Paid for each Menu Item|
+--------------------+-----------------------------+
|Chicken Tikka Masala|            35522.55000000035|
|          Pilau Rice|            34668.40000000031|
|         Bombay Aloo|            25799.19999999997|
|                Naan|                      21817.5|
|               Korma|           17533.049999999836|
|         Onion Bhaji|           15661.749999999676|
|      Butter Chicken|            15143.39999999987|
|              Madras|           14914.199999999848|
|       Plain Papadum|           14443.200000000055|
|         Garlic Naan|            14177.69999999972|
|       Mushroom Rice|           13524.799999999781|
|Chicken Tikka (Main)|           13514.499999999898|
|Tandoori Mixed Grill|           13503.499999999916|
|           Saag Aloo|            12429.54999999985|
|     Korma - Chicken|           10874.249999999949|
|     Chicken Biryani|           10656.4499999

In [0]:
restaurant_2_products_price.head(302)
restaurant_1_products_price.head(248)

Out[84]: [Row(Menu_Item='Mint Sauce', Correct_Price=0.5),
 Row(Menu_Item='Lime Pickle', Correct_Price=0.5),
 Row(Menu_Item='Mango Chutney', Correct_Price=0.5),
 Row(Menu_Item='Red Sauce', Correct_Price=0.5),
 Row(Menu_Item='Onion Chutney', Correct_Price=0.5),
 Row(Menu_Item='Plain Papadum', Correct_Price=0.8),
 Row(Menu_Item='Spicy Papadum', Correct_Price=0.8),
 Row(Menu_Item='Chapati', Correct_Price=1.95),
 Row(Menu_Item='Puree', Correct_Price=1.95),
 Row(Menu_Item='Tandoori Roti', Correct_Price=2.6),
 Row(Menu_Item='Plain Naan', Correct_Price=2.6),
 Row(Menu_Item='Raitha', Correct_Price=2.95),
 Row(Menu_Item='Keema Naan', Correct_Price=2.95),
 Row(Menu_Item='Peshwari Naan', Correct_Price=2.95),
 Row(Menu_Item='Pilau Rice', Correct_Price=2.95),
 Row(Menu_Item='Green Salad', Correct_Price=2.95),
 Row(Menu_Item='Bottle Coke', Correct_Price=2.95),
 Row(Menu_Item='Plain Rice', Correct_Price=2.95),
 Row(Menu_Item='Garlic Naan', Correct_Price=2.95),
 Row(Menu_Item='Onion Naan', Correct_Pric

- make 3 separate dataframes each to store one of (Mains, Starters and Drinks)
- so I can compare the top 5 dishes or drinks in each category between the two different restaurants based on the sales

In [0]:
starters_restaurant_1 = restaurant_1_products_price.select("Menu_Item", "Correct_Price").where(restaurant_1_products_price.Correct_Price.between(0, 7.00))
starters_restaurant_1.show()
### created starters only menu, with a few drinks inside still that need to be removed, firstly will make mains menu and drinks menu.

+-------------+-------------+
|    Menu_Item|Correct_Price|
+-------------+-------------+
|   Mint Sauce|          0.5|
|  Lime Pickle|          0.5|
|Mango Chutney|          0.5|
|    Red Sauce|          0.5|
|Onion Chutney|          0.5|
|Plain Papadum|          0.8|
|Spicy Papadum|          0.8|
|      Chapati|         1.95|
|        Puree|         1.95|
|Tandoori Roti|          2.6|
|   Plain Naan|          2.6|
|       Raitha|         2.95|
|   Keema Naan|         2.95|
|Peshwari Naan|         2.95|
|   Pilau Rice|         2.95|
|  Green Salad|         2.95|
|  Bottle Coke|         2.95|
|   Plain Rice|         2.95|
|  Garlic Naan|         2.95|
|   Onion Naan|         2.95|
+-------------+-------------+
only showing top 20 rows



In [0]:
mains_restaurant_1 = restaurant_1_products_price.select("Menu_Item", "Correct_Price").where(restaurant_1_products_price.Correct_Price.between(7.90, 12.95))
mains_restaurant_1.show()

+--------------------+-------------+
|           Menu_Item|Correct_Price|
+--------------------+-------------+
|  Chicken Chaat Main|          7.9|
|    Madras - Chicken|         7.95|
|       Madras - Lamb|         7.95|
|     Curry - Chicken|         7.95|
|  Vindaloo - Chicken|         7.95|
|Madras - Chicken ...|         7.95|
|        Curry - Lamb|         7.95|
|Curry - Chicken T...|         7.95|
|Vindaloo - Chicke...|         7.95|
|     Vindaloo - Lamb|         7.95|
|              Madras|         7.95|
|               Curry|         7.95|
|            Vindaloo|         7.95|
|     Korma - Chicken|         8.95|
|Chicken Tikka Masala|         8.95|
|Pathia - Chicken ...|         8.95|
|Chicken Tikka Jal...|         8.95|
|        Methi - Lamb|         8.95|
|Tandoori Chicken ...|         8.95|
|     Bhuna - Chicken|         8.95|
+--------------------+-------------+
only showing top 20 rows



In [0]:
drinks = ['COBRA ( LARGE )', 'COBRA (660ML)', 'Prier Water (750ml)', 'Still Water (750ml)', 'COBRA (330ml)', 'Perrier Water (750ml)', 'Coke 1.5 ltr', 'Diet Coke 1.5 ltr', 'Lemonade 1.5 ltr', 'Bottle Diet Coke', 'House Red wine 75cl', 'House white wine 75cl']
drinks_restaurant_1 = restaurant_1_products_price.filter(restaurant_1_products_price['Menu_Item'].isin(drinks))
drinks_restaurant_1.show()

+--------------------+-------------+
|           Menu_Item|Correct_Price|
+--------------------+-------------+
|    Bottle Diet Coke|         2.95|
|    Lemonade 1.5 ltr|         2.95|
|   Diet Coke 1.5 ltr|         2.95|
|        Coke 1.5 ltr|         2.95|
|Perrier Water (75...|         3.95|
| Still Water (750ml)|         3.95|
| Prier Water (750ml)|         3.95|
|     COBRA ( LARGE )|         5.95|
|       COBRA (660ML)|         5.95|
| House Red wine 75cl|        17.95|
|House white wine ...|        17.95|
+--------------------+-------------+

